In [1]:
from sfcn import SFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression
import matplotlib.pyplot as plt

2021-12-16 11:26:28.412993: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# col_list = pd.read_csv('fields/subcortical_volumes.csv')['field id'].to_list()

# columns=['path']
# for col in col_list:
#     columns.append(str(col)+'-2.0')

In [3]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid').dropna()
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid').dropna()
test_df = pd.read_csv('csv/split_test.csv', index_col='eid').dropna()

In [4]:
name = 'sfcn_vanilla'
batch_size = 5
gpu_num = 5
cpu_workers = 5
epochs_num = 64

index=2

In [5]:
input_dim = [182, 218, 182]
num_output = len(train_df.columns)-1

In [6]:
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='quantile')

scaler_instance = train_gen.get_scaler_instance()

In [7]:
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False)

In [8]:
test_gen = VolumeDataGeneratorRegression(
    sample_df=test_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False
)

In [9]:
model = SFCN(
        input_dim=[182, 218, 182, 1], 
        output_dim=num_output,
        conv_num_filters=[32, 64, 128, 256, 256, 64], 
        conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
        conv_strides=[1, 1, 1, 1, 1, 1],
        conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
        pooling_size=[2, 2, 2, 2, 2],
        pooling_type=['max_pool', 'max_pool', 'max_pool', 'max_pool', 'max_pool'],
        batch_norm=True,
        dropout=False,
        softmax=False,
        gpu_num=gpu_num,
        name=name+'_'+str(index)
        )
model.compile(learning_rate=3e-4)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


2021-12-16 11:26:45.214637: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-16 11:26:45.216460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-16 11:26:45.616670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-16 11:26:45.619744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-16 11:26:45.622748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:07:0

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [10]:
model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=epochs_num, workers=cpu_workers)

2021-12-16 11:26:55.608796: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_1696"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  va

Epoch 1/64
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


2021-12-16 11:27:11.589118: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-16 11:27:13.063012: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-16 11:27:13.344215: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-12-16 11:27:16.052339: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


1238/1238 [==============================] - ETA: 0s - loss: 0.1050 - mae: 0.2709

2021-12-16 11:39:03.271626: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_20050"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  v

1238/1238 [==============================] - 929s 657ms/step - loss: 0.1050 - mae: 0.2709 - val_loss: 0.1951 - val_mae: 0.3573
Epoch 2/64
1238/1238 [==============================] - 814s 656ms/step - loss: 0.0827 - mae: 0.2485 - val_loss: 0.0971 - val_mae: 0.2617
Epoch 3/64
1238/1238 [==============================] - 805s 649ms/step - loss: 0.0798 - mae: 0.2435 - val_loss: 0.0998 - val_mae: 0.2638
Epoch 4/64
1238/1238 [==============================] - 805s 649ms/step - loss: 0.0754 - mae: 0.2358 - val_loss: 0.1025 - val_mae: 0.2677
Epoch 5/64
1238/1238 [==============================] - 815s 657ms/step - loss: 0.0714 - mae: 0.2280 - val_loss: 0.1129 - val_mae: 0.2716
Epoch 6/64
1238/1238 [==============================] - 797s 642ms/step - loss: 0.0682 - mae: 0.2212 - val_loss: 0.0955 - val_mae: 0.2610
Epoch 7/64
1238/1238 [==============================] - 796s 642ms/step - loss: 0.0666 - mae: 0.2179 - val_loss: 0.0875 - val_mae: 0.2512
Epoch 8/64
1238/1238 [=======================

In [11]:
model.load_weights('weights/checkpoint_' + name + '_' + str(index))
model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)

2021-12-16 17:41:17.477232: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_129355"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  

# AVG pool

In [ ]:
# name = 'sfcn_avg_pool'

In [ ]:
# model = SFCN(
#         input_dim=[182, 218, 182, 1], 
#         output_dim=num_output,
#         conv_num_filters=[32, 64, 128, 256, 256, 64], 
#         conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
#         conv_strides=[1, 1, 1, 1, 1, 1],
#         conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
#         pooling_size=[2, 2, 2, 2, 2],
#         pooling_type=['avg_pool', 'avg_pool', 'avg_pool', 'avg_pool', 'avg_pool'],
#         batch_norm=True,
#         dropout=False,
#         softmax=False,
#         gpu_num=gpu_num,
#         name=name+'_'+str(0)
#         )
# model.compile(learning_rate=3e-4)

In [ ]:
# model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=40, workers=cpu_workers)

In [ ]:
# model.load_weights('weights/checkpoint_' + name + '_' + str(0))
# model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
# model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)

In [ ]:
#model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=40)

In [ ]:
#model.load_weights('weights/checkpoint_' + name)
#model.evaluate_generator(valid_gen, batch_size)

In [ ]:
# from pathlib import Path
# from datetime import date

# r2 = -0.29363421770141684
# mae = 0.27212154832328556
# mse = 0.10790403615065694
# today = date.today()

# columns = ['date','r2','mae','mse']

# # df = pd.DataFrame(entry, columns=columns, index='date')

# filedir = Path.cwd().joinpath('results')
# filedir.mkdir(parents=True, exist_ok=True)

# fn = filedir.joinpath(name +'.csv')


# if fn.exists():
#     entry ={
#         'date':today,
#         'r2':r2,
#         'mae':mae,
#         'mse':mse }
#     df = pd.read_csv(fn)    
#     df = df.append(entry, ignore_index=True)
# else:
#     df = pd.DataFrame([[today, r2, mae, mse]], columns=columns)

# df.to_csv(fn, index=False)

In [ ]:
#model.model.evaluate(valid_gen)
# train_df['path'].iloc[0]aa

In [ ]:

#data_size=200
#X = np.random.normal(loc=5, scale=5, size=(data_size, *(input_dim), 1))
#y = np.random.normal(loc=1, scale=1, size=(data_size, num_output))
#model.train(X, y, batch_size, 300)

In [ ]:
#x_test = np.random.normal(loc=8, scale=3, size=(1, 160, 192, 160, 1))

In [ ]:
#model.predict(x_test)

In [ ]:
#y_test = np.mean(x_test, axis=(1,2,3,4))
#print(y_test)